<a href="https://colab.research.google.com/github/Bookky123/Big-Data-/blob/main/Final_Project_tools.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install opendatasets -q

# **Import library**

In [ ]:
from os.path import split
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from imblearn.over_sampling import SMOTE
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC

##download data from kaggle

In [ ]:
import opendatasets as od
od.download('https://www.kaggle.com/datasets/uciml/red-wine-quality-cortez-et-al-2009')

In [ ]:
file = ('/content/red-wine-quality-cortez-et-al-2009/winequality-red.csv')
df = pd.read_csv(file)
df.sample(5)

In [ ]:
df.info()

# **Data Preprocessing**

## missing value

In [ ]:
df.isnull().sum()

In [ ]:
columns = list(df.columns)
columns

## visualization & plots

### heatmap

In [ ]:
corr = df.corr()
plt.figure(figsize=(10,8))
sns.heatmap(corr, annot=True, cmap='coolwarm', fmt='.2f', vmin=-1, vmax=1)
plt.title('Correlation Matrix')

### Scatterplot & Boxplot

In [ ]:
fig, ax = plt.subplots(11, 2, figsize=(16,45))
plt.subplots_adjust(hspace=0.5)
for i in range(11):
    sns.boxplot(x=columns[i], data=df, ax=ax[i,0])
    sns.scatterplot(x=columns[i], y='quality',data=df, ax=ax[i,1],hue='quality')

### Target categorizing

In [ ]:
df.quality.unique()

In [ ]:
df = df.replace({'quality' : {
    3 : 'bad',
    4 : 'bad',
    5 : 'middle',
    6 : 'middle',
    7 : 'good',
    8 : 'good',
}})

In [ ]:
df.head(5)

### Normalization

In [ ]:
X = df.drop(columns='quality')
y = df.quality

In [ ]:
df.describe()

# Modeling

In [ ]:
df.quality.value_counts()

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.2, random_state=42)

## SMOTE

In [ ]:
#เเยกตัวที่ต้องการ ปรับสมดุล ในกรณีนี้เราสนใจ good&middle
X_train_gm = X_train[y_train.isin(['good','middle'])]
y_train_gm = y_train[y_train.isin(['good','middle'])]

#ใช้ smote ปรับสมดุล good&middle
smote = SMOTE(random_state=42)
X_resampled, y_resampled = smote.fit_resample(X_train_gm, y_train_gm)

In [ ]:
#รวมข้อมูลที่ปรับสมดุลเเล้ว กับ bad
X_train_bad = X_train[y_train == 'bad']
y_train_bad = y_train[y_train == 'bad']

X_train_resampled = pd.concat([X_resampled, X_train_bad])
y_train_resampled = pd.concat([y_resampled, y_train_bad])
#ข้อมูลถูกปรับสมดุลเเล้ว

## RandomForestClassifier

In [ ]:
parameters = {
    'n_estimators' : [50,150,500],
    'criterion' : ['gini','entropy'],
    'max_features' : ['sqrt','log2']
}

rfc = RandomForestClassifier(n_jobs=-1)
rfc_cv = GridSearchCV(estimator=rfc, param_grid=parameters, cv=5).fit(X_train_resampled, y_train_resampled)

print(rfc_cv.best_params_)
print('accuracy_score :' ,rfc_cv.best_score_)

In [ ]:
rf = RandomForestClassifier(**rfc_cv.best_params_).fit(X_train_resampled, y_train_resampled)

#Predict model
y_pred_rf = rf.predict(X_test)

rf_score = round(rf.score(X_test,y_test),3)
print('RandomForestClassifier Score : ',rf_score)

In [ ]:
y_test.value_counts()

In [ ]:
print(classification_report(y_test, y_pred_rf))

In [ ]:
print(confusion_matrix(y_test, y_pred_rf))

### Bad & Not Bad

เเยกทำนาย ค่า Bad เพราะ ในโมเดล ไม่สามารถทำนายค่าBad ได้เลย

In [ ]:
#สร้าง columns is_bad
df['is_bad'] = df['quality'].apply(lambda x: 1 if x == 'bad' else 0)

#เเบ่ง X,y
X_bad = df.drop(columns=['quality', 'is_bad'], axis=1)
y_bad = df['is_bad']

In [ ]:
X_train_bad, X_test_bad, y_train_bad, y_test_bad = train_test_split(X_bad,y_bad, test_size=0.2, random_state=42)

In [ ]:
#สร้าง Model is bad
rf_bad = RandomForestClassifier(**rfc_cv.best_params_).fit(X_train_bad, y_train_bad)
rf_bad.fit(X_train_bad, y_train_bad)

In [ ]:
# predict
y_pred_bad = rf_bad.predict(X_test_bad)

In [ ]:
print(classification_report(y_test_bad, y_pred_bad))

In [ ]:
print(confusion_matrix(y_test_bad, y_pred_bad))

##  KNeighborsClassifier

In [ ]:
parameters = {
    'n_neighbors' : list(np.arange(3,50,2)),
    'weights' : ['uniform', 'distance'],
    'p' : [1,2,3,4]
}

knn = KNeighborsClassifier()
knn_cv = GridSearchCV(estimator=knn, param_grid=parameters, cv=10).fit(X_train_resampled,y_train_resampled)

print(knn_cv.best_params_)
print('accurracy ',knn_cv.best_score_)

In [ ]:
knn = KNeighborsClassifier(**knn_cv.best_params_).fit(X_train_resampled, y_train_resampled)

In [ ]:
y_pred_knn = knn_cv.predict(X_test)

knn_score = round(knn_cv.score(X_test,y_test), 3)
print('KNeighborsClassifier Score : ',knn_score)

In [ ]:
print(classification_report(y_test, y_pred_knn))

In [ ]:
print(confusion_matrix(y_test, y_pred_knn))

## SVC

In [ ]:
parameters = {
    'C' : [0.001, 0.01, 0.1, 1, 10, 100, 1000],
    'gamma' : [0.001, 0.01, 0.1, 1, 10, 100, 1000]
}

svc = SVC()
svc_cv = GridSearchCV(estimator=svc, param_grid=parameters, cv=10).fit(X_train_resampled,y_train_resampled)

print('Turned hyper parameter : ', svc_cv.best_params_)
print('accurracy score : ', svc_cv.best_score_)

In [ ]:
svc = SVC(**svc_cv.best_params_).fit(X_train_resampled, y_train_resampled)

In [ ]:
y_pred_svc = svc_cv.predict(X_test)

svc_score = round(svc.score(X_test, y_test),3)
print('SVC Score : ', svc_score)

In [ ]:
print(classification_report(y_test, y_pred_svc))

In [ ]:
print(confusion_matrix(y_test, y_pred_svc))

# Result

In [ ]:
result = pd.DataFrame({
    'Model' : ['RandomForestClassifier', 'KNeighborsClassifier','SVC'],
    'Score' : [rf_score, knn_score, svc_score]
})

result.sort_values(by='Score', inplace=True, ascending=True)

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(15,5))

sns.barplot(x='Model',y='Score', data=result)
ax.bar_label(ax.containers[0])
ax.set_xticklabels(labels=result.Model, rotation=300)
plt.show()

**The Best Score : RandomForestClassifier**

In [ ]:
rfc = RandomForestClassifier(**rfc_cv.best_params_)

In [ ]:
rfc.fit(X,y)